<a href="https://colab.research.google.com/github/SuriyaPriya17/image_processing/blob/main/siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
import numpy as np

# 1. Data Preparation - Create pairs of images
def make_pairs(images, labels):
    pair_images = []
    pair_labels = []
    num_classes = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(num_classes)]

    for anchor_idx in range(len(images)):
        current_image = images[anchor_idx]
        current_label = labels[anchor_idx]

        # Positive pair
        positive_idx = np.random.choice(idx[current_label])
        while positive_idx == anchor_idx:
            positive_idx = np.random.choice(idx[current_label])
        positive_image = images[positive_idx]

        pair_images.append([current_image, positive_image])
        pair_labels.append(1)

        # Negative pair
        negative_label = np.random.randint(0, num_classes)
        while negative_label == current_label:
            negative_label = np.random.randint(0, num_classes)
        negative_image = images[np.random.choice(idx[negative_label])]

        pair_images.append([current_image, negative_image])
        pair_labels.append(0)

    return np.array(pair_images), np.array(pair_labels)


# 2. Build the Siamese Network with Transfer Learning
def build_siamese_model(input_shape, embedding_dim=128):
    base_cnn = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_cnn.trainable = False

    input_tensor = keras.Input(shape=input_shape)
    x = base_cnn(input_tensor, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(embedding_dim, activation="relu")(x)
    embedding_network = keras.Model(input_tensor, x, name="embedding_network")

    # Define Siamese inputs
    input_a = keras.Input(shape=input_shape, name="anchor")
    input_p = keras.Input(shape=input_shape, name="positive")
    input_n = keras.Input(shape=input_shape, name="negative")

    # Get embeddings
    embedding_a = embedding_network(input_a)
    embedding_p = embedding_network(input_p)
    embedding_n = embedding_network(input_n)

    # Use a Lambda layer to stack embeddings safely
    output = layers.Lambda(lambda tensors: tf.stack(tensors, axis=1))([embedding_a, embedding_p, embedding_n])

    siamese_model = keras.Model(inputs=[input_a, input_p, input_n], outputs=output, name="siamese_model")
    return siamese_model, embedding_network


# 3. Define Triplet Loss
def triplet_loss(y_true, y_pred, margin=1.0):
    anchor, positive, negative = tf.unstack(y_pred, num=3, axis=1)
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    loss = tf.maximum(pos_dist - neg_dist + margin, 0.0)
    return tf.reduce_mean(loss)


# Example Usage
if __name__ == "__main__":
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
    x_train = x_train[:1000]
    y_train = y_train[:1000]
    x_test = x_test[:200]
    y_test = y_test[:200]

    input_shape = (32, 32, 3)
    x_train = tf.image.resize(x_train, (input_shape[0], input_shape[1])) / 255.0
    x_test = tf.image.resize(x_test, (input_shape[0], input_shape[1])) / 255.0
    y_train = y_train.flatten()
    y_test = y_test.flatten()

    siamese_model, embedding_network = build_siamese_model(input_shape)

    siamese_model.compile(optimizer=keras.optimizers.Adam(1e-4),
                          loss=triplet_loss)

    # Dummy triplets for demonstration
    dummy_anchor = x_train[:400]
    # Create dummy_positive and dummy_negative with the same number of samples as dummy_anchor
    dummy_positive = x_train[100:500]
    dummy_negative = x_train[200:600]
    dummy_y = np.zeros(len(dummy_anchor))

    print("Training the Siamese network (dummy data)...")
    siamese_model.fit([dummy_anchor, dummy_positive, dummy_negative],
                      dummy_y, epochs=1)

    print("\nEvaluating similarity with the trained embedding network...")
    img1 = x_test[0]
    img2_same_class = x_test[1]
    img3_diff_class = x_test[2]

    emb1 = embedding_network.predict(np.expand_dims(img1, axis=0))
    emb2 = embedding_network.predict(np.expand_dims(img2_same_class, axis=0))
    emb3 = embedding_network.predict(np.expand_dims(img3_diff_class, axis=0))

    similarity_same = tf.keras.losses.cosine_similarity(emb1, emb2).numpy()
    similarity_diff = tf.keras.losses.cosine_similarity(emb1, emb3).numpy()

    print(f"Similarity between img1 and img2 (same class): {similarity_same}")
    print(f"Similarity between img1 and img3 (different class): {similarity_diff}")

Training the Siamese network (dummy data)...
13/13 ━━━━━━━━━━━━━━━━━━━━ 23s 436ms/step - loss: 1.2532

Evaluating similarity with the trained embedding network...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Similarity between img1 and img2 (same class): [-0.9966147]
Similarity between img1 and img3 (different class): [-0.9970206]
